In [14]:
import sympy as sp

def finde_extremwert(ziel_funktion_expr, nebenbedingung_eq, zu_eliminierende_variable, verbleibende_variable):
    """
    NEUE, VERBESSERTE VERSION: Löst ein Extremwertproblem und gibt die Ergebnisse
    in einem Dictionary zurück, anstatt sie zu drucken. Diese Funktion ist
    wirklich allgemein und muss nicht mehr angepasst werden.
    """
    try:
        # Nebenbedingung auflösen
        ersatz_formel_loesungen = sp.solve(nebenbedingung_eq, zu_eliminierende_variable)
        if not ersatz_formel_loesungen:
            return {'status': 'fehler', 'info': 'Nebenbedingung nicht auflösbar'}
        ersatz_formel = ersatz_formel_loesungen[0]

        # In Zielfunktion einsetzen
        ziel_funktion_final = ziel_funktion_expr.subs(zu_eliminierende_variable, ersatz_formel)

        # Ableiten und kritische Punkte finden
        erste_ableitung = sp.diff(ziel_funktion_final, verbleibende_variable)
        # Wir finden alle reellen Punkte, die physische Interpretation erfolgt später.
        kritische_punkte = sp.solveset(erste_ableitung, verbleibende_variable, domain=sp.Reals)

        # Zweiten Ableitung für die Prüfung vorbereiten
        zweite_ableitung = sp.diff(erste_ableitung, verbleibende_variable)
        
        ergebnisse = []
        for p in kritische_punkte:
            # Wir betrachten nur reelle, numerische Lösungen
            if p.is_real and p.is_number and p >= 0: # Annahme: physikalische Grössen sind nicht negativ
                wert_2_abl = zweite_ableitung.subs(verbleibende_variable, p)
                
                typ = 'unbekannt'
                if wert_2_abl < 0: typ = 'Maximum'
                if wert_2_abl > 0: typ = 'Minimum'
                
                if typ != 'unbekannt':
                    wert_verbleibend = p
                    wert_eliminiert = ersatz_formel.subs(verbleibende_variable, p)
                    ergebnisse.append({
                        'status': 'erfolg',
                        'typ': typ,
                        'variable_verbleibend': verbleibende_variable,
                        'wert_verbleibend': wert_verbleibend,
                        'variable_eliminiert': zu_eliminierende_variable,
                        'wert_eliminiert': wert_eliminiert
                    })
        return ergebnisse

    except Exception as e:
        return {'status': 'fehler', 'info': str(e)}


if __name__ == "__main__":
    
    print("="*40)
    print("  Beispiel: Maximales Rechteck unter der Parabel")
    print("="*40)
    
    # Schritt 1: Das Problem mathematisch formulieren
    x, h = sp.symbols('x h')
    # Ziel: Fläche A = (Breite) * (Höhe) = (2*x) * h maximieren
    flaeche_A = 2 * x * h
    # Einschränkung: Die Höhe h wird durch die Parabel y = 4 - x^2 bestimmt
    parabel_gleichung = sp.Eq(h, 4 - x**2)
    
    # Schritt 2: Die allgemeine Funktion aufrufen, um die Rohdaten zu erhalten
    loesungen = finde_extremwert(
        ziel_funktion_expr=flaeche_A,
        nebenbedingung_eq=parabel_gleichung,
        zu_eliminierende_variable=h,
        verbleibende_variable=x
    )
    
    # Schritt 3: Die Ergebnisse interpretieren und die Frage beantworten
    print("\n--- FINALES ERGEBNIS (interpretiert) ---")
    
    if loesungen:
        # Wir suchen nach einem Maximum
        max_loesung = next((l for l in loesungen if l['typ'] == 'Maximum'), None)
        
        if max_loesung:
            # Rohwerte aus dem Ergebnis-Dictionary holen
            optimale_hoehe = max_loesung['wert_eliminiert']
            halbe_breite_x = max_loesung['wert_verbleibend']
            
            # Problemspezifische Berechnung: Die Gesamtbreite ist 2*x
            optimale_breite = 2 * halbe_breite_x
            
            print(f"Die Aufgabe verlangt ein Maximum. Passende Lösung gefunden:")
            print(f"Die optimale Höhe (h) ist: {optimale_hoehe.evalf(4)} (Exakt: {optimale_hoehe})")
            print(f"Die optimale Breite (2*x) ist: {optimale_breite.evalf(4)} (Exakt: {optimale_breite})")
        else:
            print("Es konnte kein Maximum gefunden werden.")
    else:
        print("Es konnte keine Lösung gefunden werden.")

## Beispiel 2
    print("="*40)
    print("  Beispiel: Weidezaun am Fluss")
    print("="*40)
    
    # Schritt 1: Das Problem mathematisch formulieren
    l, b = sp.symbols('l b') # l für Länge, b für Breite
    # Ziel: Fläche A = l * b maximieren
    flaeche = l * b
    # Einschränkung: Zaunlänge l + 2*b = 100
    umfang_gleichung = sp.Eq(l + 2*b, 100)
    
    # Schritt 2: Die allgemeine Funktion aufrufen
    loesungen = finde_extremwert(
        ziel_funktion_expr=flaeche,
        nebenbedingung_eq=umfang_gleichung,
        zu_eliminierende_variable=l, # 'l' lässt sich leicht eliminieren
        verbleibende_variable=b     # 'b' bleibt übrig
    )
    
    # Schritt 3: Die Ergebnisse interpretieren und die Frage beantworten
    print("\n--- FINALES ERGEBNIS (interpretiert) ---")
    
    if loesungen:
        # Wir suchen nach einem Maximum
        max_loesung = next((l for l in loesungen if l['typ'] == 'Maximum'), None)
        
        if max_loesung:
            optimale_breite = max_loesung['wert_verbleibend']
            optimale_laenge = max_loesung['wert_eliminiert']
            
            # Die maximale Fläche berechnen
            max_flaeche = optimale_laenge * optimale_breite
            
            print(f"Die Aufgabe verlangt ein Maximum. Passende Lösung gefunden:")
            print(f"Die optimale Breite (b) beträgt: {optimale_breite} Meter")
            print(f"Die optimale Länge (l) beträgt: {optimale_laenge} Meter")
            print(f"Die daraus resultierende maximale Fläche beträgt: {max_flaeche} Quadratmeter")
        else:
            print("Es konnte kein Maximum gefunden werden.")
    else:
        print("Es konnte keine Lösung gefunden werden.")

  Beispiel: Maximales Rechteck unter der Parabel

--- FINALES ERGEBNIS (interpretiert) ---
Die Aufgabe verlangt ein Maximum. Passende Lösung gefunden:
Die optimale Höhe (h) ist: 2.667 (Exakt: 8/3)
Die optimale Breite (2*x) ist: 2.309 (Exakt: 4*sqrt(3)/3)
  Beispiel: Weidezaun am Fluss

--- FINALES ERGEBNIS (interpretiert) ---
Die Aufgabe verlangt ein Maximum. Passende Lösung gefunden:
Die optimale Breite (b) beträgt: 25 Meter
Die optimale Länge (l) beträgt: 50 Meter
Die daraus resultierende maximale Fläche beträgt: 1250 Quadratmeter
